In [1]:
#从源数据文件提取并处理为18个指标：高压、低压以及合计新装增容、新装（户数、容量），然后加工出高压、低压以及合计增容（户数、容量）
#该块代码转为处理新装增容数据

import pandas as pd
import os
import re
from pathlib import Path
from typing import List, Dict, Optional
import traceback

# ==============================================================================
# 1. 配置层 (Configuration Layer)
# ==============================================================================
class Config:
    """集中管理所有可配置参数"""
    INPUT_DIR = Path(r'E:\A智网\业扩分析\数据准备\新装增容数据')
    OUTPUT_FILE = Path(r'E:\A智网\业扩分析\12月分析\11月业扩月度报告\25年11月业扩报告_新装增容业扩.xlsx') # 建议新文件名以防覆盖
    FILENAME_PATTERN = r'^\d{6}\.xlsx$'

    TARGET_SHEET_NAME = '国网湖北省电力有限公司'
    SKIP_ROWS = 3
    HEADER_ROWS_COUNT = 4

    ### 重构/新增开始 ###

    # --- 核心指标的最终命名 ---
    # Part 1: 从源文件直接提取的指标 (栏目 13-24)

    # 1.1 完成新装增容 (合计)
    COL_TOTAL_ALL_COUNT     = '完成新装增容_户数'
    COL_TOTAL_ALL_CAPACITY  = '完成新装增容_容量'

    # 1.2 完成新装增容 (高压)
    COL_TOTAL_HV_COUNT      = '高压完成新装增容_户数'
    COL_TOTAL_HV_CAPACITY   = '高压完成新装增容_容量'

    # 1.3 完成新装增容 (低压)
    COL_TOTAL_LV_COUNT      = '低压完成新装增容_户数'
    COL_TOTAL_LV_CAPACITY   = '低压完成新装增容_容量'

    # 1.4 完成新装 (合计)
    COL_NEW_ALL_COUNT       = '完成新装_户数'
    COL_NEW_ALL_CAPACITY    = '完成新装_容量'

    # 1.5 完成新装 (高压)
    COL_NEW_HV_COUNT        = '高压完成新装_户数'
    COL_NEW_HV_CAPACITY     = '高压完成新装_容量'

    # 1.6 完成新装 (低压)
    COL_NEW_LV_COUNT        = '低压完成新装_户数'
    COL_NEW_LV_CAPACITY     = '低压完成新装_容量'


    # Part 2: 通过计算生成的衍生指标 (增容 = 新装增容 - 新装)

    # 2.1 完成增容 (合计)
    COL_INC_ALL_COUNT       = '完成增容_户数'
    COL_INC_ALL_CAPACITY    = '完成增容_容量'

    # 2.2 完成增容 (高压)
    COL_INC_HV_COUNT        = '高压完成增容_户数'
    COL_INC_HV_CAPACITY     = '高压完成增容_容量'

    # 2.3 完成增容 (低压)
    COL_INC_LV_COUNT        = '低压完成增容_户数'
    COL_INC_LV_CAPACITY     = '低压完成增容_容量'

    ### 重构/新增结束 ###

    @staticmethod
    def get_sheets_to_create() -> Dict[str, str]:
        """
        定义要创建的所有工作表。
        字典的键 (key) 是最终Excel中的工作表名称。
        字典的值 (value) 是 long_df 中对应的数据列名。
        此方法现在将返回所有18个指标的工作表定义。
        """
        ### 重构/新增开始 ###
        sheets = {
            # 新装增容 (合计)
            Config.COL_TOTAL_ALL_COUNT:    Config.COL_TOTAL_ALL_COUNT,
            Config.COL_TOTAL_ALL_CAPACITY: Config.COL_TOTAL_ALL_CAPACITY,
            # 新装增容 (高压)
            Config.COL_TOTAL_HV_COUNT:     Config.COL_TOTAL_HV_COUNT,
            Config.COL_TOTAL_HV_CAPACITY:  Config.COL_TOTAL_HV_CAPACITY,
            # 新装增容 (低压)
            Config.COL_TOTAL_LV_COUNT:     Config.COL_TOTAL_LV_COUNT,
            Config.COL_TOTAL_LV_CAPACITY:  Config.COL_TOTAL_LV_CAPACITY,

            # 新装 (合计)
            Config.COL_NEW_ALL_COUNT:      Config.COL_NEW_ALL_COUNT,
            Config.COL_NEW_ALL_CAPACITY:   Config.COL_NEW_ALL_CAPACITY,
            # 新装 (高压)
            Config.COL_NEW_HV_COUNT:       Config.COL_NEW_HV_COUNT,
            Config.COL_NEW_HV_CAPACITY:    Config.COL_NEW_HV_CAPACITY,
            # 新装 (低压)
            Config.COL_NEW_LV_COUNT:       Config.COL_NEW_LV_COUNT,
            Config.COL_NEW_LV_CAPACITY:    Config.COL_NEW_LV_CAPACITY,

            # 增容 (合计) - 衍生
            Config.COL_INC_ALL_COUNT:      Config.COL_INC_ALL_COUNT,
            Config.COL_INC_ALL_CAPACITY:   Config.COL_INC_ALL_CAPACITY,
            # 增容 (高压) - 衍生
            Config.COL_INC_HV_COUNT:       Config.COL_INC_HV_COUNT,
            Config.COL_INC_HV_CAPACITY:    Config.COL_INC_HV_CAPACITY,
            # 增容 (低压) - 衍生
            Config.COL_INC_LV_COUNT:       Config.COL_INC_LV_COUNT,
            Config.COL_INC_LV_CAPACITY:    Config.COL_INC_LV_CAPACITY,
        }
        return sheets
        ### 重构/新增结束 ###

# ==============================================================================
# 2. 提取与解析层 (Extraction & Parsing Layer)
# ==============================================================================
def robust_to_numeric(series: pd.Series) -> pd.Series:
    """
    一个极其健壮的函数，用于清洗并转换一列可能包含文本格式数字的数据。
    它能处理千位分隔符(,)、首尾空格等常见问题。
    """
    s = series.astype(str).str.replace(',', '', regex=False).str.strip()
    return pd.to_numeric(s, errors='coerce')


def process_single_file(file_path: Path) -> Optional[pd.DataFrame]:
    """读取并处理单个Excel文件，返回一个干净的DataFrame"""
    year_month = file_path.stem
    print(f"  -> 正在处理文件: {file_path.name}")
    try:
        df = pd.read_excel(
            file_path, sheet_name=Config.TARGET_SHEET_NAME,
            skiprows=Config.SKIP_ROWS + Config.HEADER_ROWS_COUNT,
            header=None,
            dtype=str
        )

        df.rename(columns={0: '序号', 1: '行业'}, inplace=True)
        df = df[df['行业'].astype(str).str.strip() != '栏目'].copy()
        df.dropna(how='all', subset=df.columns[2:], inplace=True)
        df['序号'] = robust_to_numeric(df['序号'])
        df.dropna(subset=['序号'], inplace=True)
        df['序号'] = df['序号'].astype(int)
        df['年月'] = year_month

        df_final = df[['序号', '行业', '年月']].copy()

        ### 重构/新增开始 ###
        # 根据新需求，提取栏目13至24的数据
        # 换算公式： Pandas索引 = Excel列号 - 1
        # 原始数据栏目号从1开始，所以 Pandas索引 = 原始栏目号 + 1 (此为原代码逻辑，我们遵循它)

        # 提取 完成新装增容 数据
        df_final[Config.COL_TOTAL_ALL_COUNT]    = robust_to_numeric(df.iloc[:, 13+1]) # 栏目13
        df_final[Config.COL_TOTAL_ALL_CAPACITY] = robust_to_numeric(df.iloc[:, 14+1]) # 栏目14
        df_final[Config.COL_TOTAL_HV_COUNT]     = robust_to_numeric(df.iloc[:, 15+1]) # 栏目15
        df_final[Config.COL_TOTAL_HV_CAPACITY]  = robust_to_numeric(df.iloc[:, 16+1]) # 栏目16
        df_final[Config.COL_TOTAL_LV_COUNT]     = robust_to_numeric(df.iloc[:, 17+1]) # 栏目17
        df_final[Config.COL_TOTAL_LV_CAPACITY]  = robust_to_numeric(df.iloc[:, 18+1]) # 栏目18

        # 提取 完成新装 数据
        df_final[Config.COL_NEW_ALL_COUNT]      = robust_to_numeric(df.iloc[:, 19+1]) # 栏目19
        df_final[Config.COL_NEW_ALL_CAPACITY]   = robust_to_numeric(df.iloc[:, 20+1]) # 栏目20
        df_final[Config.COL_NEW_HV_COUNT]       = robust_to_numeric(df.iloc[:, 21+1]) # 栏目21
        df_final[Config.COL_NEW_HV_CAPACITY]    = robust_to_numeric(df.iloc[:, 22+1]) # 栏目22
        df_final[Config.COL_NEW_LV_COUNT]       = robust_to_numeric(df.iloc[:, 23+1]) # 栏目23
        df_final[Config.COL_NEW_LV_CAPACITY]    = robust_to_numeric(df.iloc[:, 24+1]) # 栏目24
        ### 重构/新增结束 ###

        return df_final
    except Exception as e:
        print(f"    [错误] 处理文件 {file_path.name} 时发生未知错误。")
        print(traceback.format_exc())
        return None

# ==============================================================================
# 3. 主流程 (Transformation & Loading)
# ==============================================================================
def main():
    print("开始执行数据处理流程...")
    all_dataframes = []
    if not Config.INPUT_DIR.exists():
        print(f"[致命错误] 输入文件夹不存在: {Config.INPUT_DIR}")
        return
    for file_path in sorted(Config.INPUT_DIR.glob('*.xlsx')):
        if re.match(Config.FILENAME_PATTERN, file_path.name):
            df = process_single_file(file_path)
            if df is not None:
                all_dataframes.append(df)
    if not all_dataframes:
        print("\n处理结束，但未从任何文件中成功提取数据。请检查上述错误日志。")
        return
    print("\n所有文件数据读取完毕，正在聚合成总表...")
    long_df = pd.concat(all_dataframes, ignore_index=True)

    long_df.fillna(0, inplace=True)

    ### 重构/新增开始 ###
    # 在聚合后的 long_df 上执行计算，生成所有“增容”相关的衍生指标
    print("正在计算所有衍生指标 (增容 = 新装增容 - 新装)...")

    # 计算 合计增容
    long_df[Config.COL_INC_ALL_COUNT]    = long_df[Config.COL_TOTAL_ALL_COUNT] - long_df[Config.COL_NEW_ALL_COUNT]
    long_df[Config.COL_INC_ALL_CAPACITY] = long_df[Config.COL_TOTAL_ALL_CAPACITY] - long_df[Config.COL_NEW_ALL_CAPACITY]

    # 计算 高压增容
    long_df[Config.COL_INC_HV_COUNT]     = long_df[Config.COL_TOTAL_HV_COUNT] - long_df[Config.COL_NEW_HV_COUNT]
    long_df[Config.COL_INC_HV_CAPACITY]  = long_df[Config.COL_TOTAL_HV_CAPACITY] - long_df[Config.COL_NEW_HV_CAPACITY]

    # 计算 低压增容
    long_df[Config.COL_INC_LV_COUNT]     = long_df[Config.COL_TOTAL_LV_COUNT] - long_df[Config.COL_NEW_LV_COUNT]
    long_df[Config.COL_INC_LV_CAPACITY]  = long_df[Config.COL_TOTAL_LV_CAPACITY] - long_df[Config.COL_NEW_LV_CAPACITY]
    ### 重构/新增结束 ###

    print("正在生成最终的多工作表Excel报表...")
    Config.OUTPUT_FILE.parent.mkdir(parents=True, exist_ok=True)
    with pd.ExcelWriter(Config.OUTPUT_FILE, engine='openpyxl') as writer:
        sheets_to_create = Config.get_sheets_to_create()
        # 现在的 sheets_to_create 已经包含了全部18个指标的工作表
        for sheet_name, value_col in sheets_to_create.items():
            print(f"  -> 正在生成工作表: {sheet_name}")
            try:
                pivot_df = long_df.pivot_table(
                    index=['序号', '行业'], columns='年月', values=value_col, aggfunc='sum')
                # 填充透视表中可能出现的NaN（例如某个月份某个行业没有数据）
                pivot_df.fillna(0, inplace=True)
                pivot_df.columns.name = None
                pivot_df = pivot_df.reset_index()
                pivot_df.rename(columns={'行业': '分类'}, inplace=True)
                pivot_df.to_excel(writer, sheet_name=sheet_name, index=False)
            except KeyError as e:
                print(f"    [警告] 在生成工作表 '{sheet_name}' 时找不到数据列 '{e}'，已跳过。")
                continue
    print(f"\n全部任务成功完成！结果已保存到: {Config.OUTPUT_FILE}")

if __name__ == '__main__':
    main()

开始执行数据处理流程...
  -> 正在处理文件: 201801.xlsx
  -> 正在处理文件: 201802.xlsx
  -> 正在处理文件: 201803.xlsx
  -> 正在处理文件: 201804.xlsx
  -> 正在处理文件: 201805.xlsx
  -> 正在处理文件: 201806.xlsx
  -> 正在处理文件: 201807.xlsx
  -> 正在处理文件: 201808.xlsx
  -> 正在处理文件: 201809.xlsx
  -> 正在处理文件: 201810.xlsx
  -> 正在处理文件: 201811.xlsx
  -> 正在处理文件: 201812.xlsx
  -> 正在处理文件: 201901.xlsx
  -> 正在处理文件: 201902.xlsx
  -> 正在处理文件: 201903.xlsx
  -> 正在处理文件: 201904.xlsx
  -> 正在处理文件: 201905.xlsx
  -> 正在处理文件: 201906.xlsx
  -> 正在处理文件: 201907.xlsx
  -> 正在处理文件: 201908.xlsx
  -> 正在处理文件: 201909.xlsx
  -> 正在处理文件: 201910.xlsx
  -> 正在处理文件: 201911.xlsx
  -> 正在处理文件: 201912.xlsx
  -> 正在处理文件: 202001.xlsx
  -> 正在处理文件: 202002.xlsx
  -> 正在处理文件: 202003.xlsx
  -> 正在处理文件: 202004.xlsx
  -> 正在处理文件: 202005.xlsx
  -> 正在处理文件: 202006.xlsx
  -> 正在处理文件: 202007.xlsx
  -> 正在处理文件: 202008.xlsx
  -> 正在处理文件: 202009.xlsx
  -> 正在处理文件: 202010.xlsx
  -> 正在处理文件: 202011.xlsx
  -> 正在处理文件: 202012.xlsx
  -> 正在处理文件: 202101.xlsx
  -> 正在处理文件: 202102.xlsx
  -> 正在处理文件: 202103.xlsx
  -> 正在处理文件

d:\Software\Anaconda\envs\dianli\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
d:\Software\Anaconda\envs\dianli\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
d:\Software\Anaconda\envs\dianli\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


  -> 正在处理文件: 202404.xlsx
  -> 正在处理文件: 202405.xlsx
  -> 正在处理文件: 202406.xlsx
  -> 正在处理文件: 202407.xlsx
  -> 正在处理文件: 202408.xlsx
  -> 正在处理文件: 202409.xlsx
  -> 正在处理文件: 202410.xlsx


d:\Software\Anaconda\envs\dianli\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


  -> 正在处理文件: 202411.xlsx
  -> 正在处理文件: 202412.xlsx
  -> 正在处理文件: 202501.xlsx
  -> 正在处理文件: 202502.xlsx
  -> 正在处理文件: 202503.xlsx


...

  -> 正在处理文件: 202504.xlsx
  -> 正在处理文件: 202505.xlsx
  -> 正在处理文件: 202506.xlsx
  -> 正在处理文件: 202507.xlsx
  -> 正在处理文件: 202508.xlsx
  -> 正在处理文件: 202509.xlsx


d:\Software\Anaconda\envs\dianli\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


  -> 正在处理文件: 202510.xlsx
  -> 正在处理文件: 202511.xlsx


d:\Software\Anaconda\envs\dianli\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")



所有文件数据读取完毕，正在聚合成总表...
正在计算所有衍生指标 (增容 = 新装增容 - 新装)...
正在生成最终的多工作表Excel报表...
  -> 正在生成工作表: 完成新装增容_户数
  -> 正在生成工作表: 完成新装增容_容量
  -> 正在生成工作表: 高压完成新装增容_户数
  -> 正在生成工作表: 高压完成新装增容_容量
  -> 正在生成工作表: 低压完成新装增容_户数
  -> 正在生成工作表: 低压完成新装增容_容量
  -> 正在生成工作表: 完成新装_户数
  -> 正在生成工作表: 完成新装_容量
  -> 正在生成工作表: 高压完成新装_户数
  -> 正在生成工作表: 高压完成新装_容量
  -> 正在生成工作表: 低压完成新装_户数
  -> 正在生成工作表: 低压完成新装_容量
  -> 正在生成工作表: 完成增容_户数
  -> 正在生成工作表: 完成增容_容量
  -> 正在生成工作表: 高压完成增容_户数
  -> 正在生成工作表: 高压完成增容_容量
  -> 正在生成工作表: 低压完成增容_户数
  -> 正在生成工作表: 低压完成增容_容量

全部任务成功完成！结果已保存到: E:\A智网\业扩分析\12月分析\11月业扩月度报告\25年11月业扩报告_新装增容业扩.xlsx


In [2]:
#从源数据文件提取并处理为18个指标：高压、低压以及合计减容销户、销户（户数、容量），然后加工出高压、低压以及合计减容（户数、容量）
#该块代码转为处理减容销户数据

import pandas as pd
import os
import re
from pathlib import Path
from typing import List, Dict, Optional
import traceback

# ==============================================================================
# 1. 配置层 (Configuration Layer)
# ==============================================================================
class Config:
    """集中管理所有可配置参数"""
    INPUT_DIR = Path(r'E:\A智网\业扩分析\数据准备\减容销户数据')
    OUTPUT_FILE = Path(r'E:\A智网\业扩分析\12月分析\11月业扩月度报告\25年11月业扩报告_减容销户业扩.xlsx') # 建议新文件名以防覆盖
    FILENAME_PATTERN = r'^\d{6}\.xlsx$'

    TARGET_SHEET_NAME = '国网湖北省电力有限公司'
    SKIP_ROWS = 3
    HEADER_ROWS_COUNT = 4

    ### 重构/新增开始 ###

    # --- 核心指标的最终命名 ---
    # Part 1: 从源文件直接提取的指标 (栏目 13-24)

    # 1.1 完成减容销户 (合计)
    COL_TOTAL_ALL_COUNT     = '完成减容销户_户数'
    COL_TOTAL_ALL_CAPACITY  = '完成减容销户_容量'

    # 1.2 完成减容销户 (高压)
    COL_TOTAL_HV_COUNT      = '高压完成减容销户_户数'
    COL_TOTAL_HV_CAPACITY   = '高压完成减容销户_容量'

    # 1.3 完成减容销户 (低压)
    COL_TOTAL_LV_COUNT      = '低压完成减容销户_户数'
    COL_TOTAL_LV_CAPACITY   = '低压完成减容销户_容量'

    # 1.4 完成销户 (合计)
    COL_NEW_ALL_COUNT       = '完成销户_户数'
    COL_NEW_ALL_CAPACITY    = '完成销户_容量'

    # 1.5 完成销户 (高压)
    COL_NEW_HV_COUNT        = '高压完成销户_户数'
    COL_NEW_HV_CAPACITY     = '高压完成销户_容量'

    # 1.6 完成销户 (低压)
    COL_NEW_LV_COUNT        = '低压完成销户_户数'
    COL_NEW_LV_CAPACITY     = '低压完成销户_容量'


    # Part 2: 通过计算生成的衍生指标 (减容 = 减容销户 - 销户)

    # 2.1 完成减容 (合计)
    COL_INC_ALL_COUNT       = '完成减容_户数'
    COL_INC_ALL_CAPACITY    = '完成减容_容量'

    # 2.2 完成减容 (高压)
    COL_INC_HV_COUNT        = '高压完成减容_户数'
    COL_INC_HV_CAPACITY     = '高压完成减容_容量'

    # 2.3 完成减容 (低压)
    COL_INC_LV_COUNT        = '低压完成减容_户数'
    COL_INC_LV_CAPACITY     = '低压完成减容_容量'

    ### 重构/新增结束 ###

    @staticmethod
    def get_sheets_to_create() -> Dict[str, str]:
        """
        定义要创建的所有工作表。
        字典的键 (key) 是最终Excel中的工作表名称。
        字典的值 (value) 是 long_df 中对应的数据列名。
        此方法现在将返回所有18个指标的工作表定义。
        """
        ### 重构/新增开始 ###
        sheets = {
            # 减容销户 (合计)
            Config.COL_TOTAL_ALL_COUNT:    Config.COL_TOTAL_ALL_COUNT,
            Config.COL_TOTAL_ALL_CAPACITY: Config.COL_TOTAL_ALL_CAPACITY,
            # 减容销户 (高压)
            Config.COL_TOTAL_HV_COUNT:     Config.COL_TOTAL_HV_COUNT,
            Config.COL_TOTAL_HV_CAPACITY:  Config.COL_TOTAL_HV_CAPACITY,
            # 减容销户 (低压)
            Config.COL_TOTAL_LV_COUNT:     Config.COL_TOTAL_LV_COUNT,
            Config.COL_TOTAL_LV_CAPACITY:  Config.COL_TOTAL_LV_CAPACITY,

            # 销户 (合计)
            Config.COL_NEW_ALL_COUNT:      Config.COL_NEW_ALL_COUNT,
            Config.COL_NEW_ALL_CAPACITY:   Config.COL_NEW_ALL_CAPACITY,
            # 销户 (高压)
            Config.COL_NEW_HV_COUNT:       Config.COL_NEW_HV_COUNT,
            Config.COL_NEW_HV_CAPACITY:    Config.COL_NEW_HV_CAPACITY,
            # 销户 (低压)
            Config.COL_NEW_LV_COUNT:       Config.COL_NEW_LV_COUNT,
            Config.COL_NEW_LV_CAPACITY:    Config.COL_NEW_LV_CAPACITY,

            # 减容 (合计) - 衍生
            Config.COL_INC_ALL_COUNT:      Config.COL_INC_ALL_COUNT,
            Config.COL_INC_ALL_CAPACITY:   Config.COL_INC_ALL_CAPACITY,
            # 减容 (高压) - 衍生
            Config.COL_INC_HV_COUNT:       Config.COL_INC_HV_COUNT,
            Config.COL_INC_HV_CAPACITY:    Config.COL_INC_HV_CAPACITY,
            # 减容 (低压) - 衍生
            Config.COL_INC_LV_COUNT:       Config.COL_INC_LV_COUNT,
            Config.COL_INC_LV_CAPACITY:    Config.COL_INC_LV_CAPACITY,
        }
        return sheets
        ### 重构/新增结束 ###

# ==============================================================================
# 2. 提取与解析层 (Extraction & Parsing Layer)
# ==============================================================================
def robust_to_numeric(series: pd.Series) -> pd.Series:
    """
    一个极其健壮的函数，用于清洗并转换一列可能包含文本格式数字的数据。
    它能处理千位分隔符(,)、首尾空格等常见问题。
    """
    s = series.astype(str).str.replace(',', '', regex=False).str.strip()
    return pd.to_numeric(s, errors='coerce')


def process_single_file(file_path: Path) -> Optional[pd.DataFrame]:
    """读取并处理单个Excel文件，返回一个干净的DataFrame"""
    year_month = file_path.stem
    print(f"  -> 正在处理文件: {file_path.name}")
    try:
        df = pd.read_excel(
            file_path, sheet_name=Config.TARGET_SHEET_NAME,
            skiprows=Config.SKIP_ROWS + Config.HEADER_ROWS_COUNT,
            header=None,
            dtype=str
        )

        df.rename(columns={0: '序号', 1: '行业'}, inplace=True)
        df = df[df['行业'].astype(str).str.strip() != '栏目'].copy()
        df.dropna(how='all', subset=df.columns[2:], inplace=True)
        df['序号'] = robust_to_numeric(df['序号'])
        df.dropna(subset=['序号'], inplace=True)
        df['序号'] = df['序号'].astype(int)
        df['年月'] = year_month

        df_final = df[['序号', '行业', '年月']].copy()

        ### 重构/新增开始 ###
        # 根据新需求，提取栏目13至24的数据
        # 换算公式： Pandas索引 = Excel列号 - 1
        # 原始数据栏目号从1开始，所以 Pandas索引 = 原始栏目号 + 1 (此为原代码逻辑，我们遵循它)

        # 提取 完成减容销户 数据
        df_final[Config.COL_TOTAL_ALL_COUNT]    = robust_to_numeric(df.iloc[:, 13+1]) # 栏目13
        df_final[Config.COL_TOTAL_ALL_CAPACITY] = robust_to_numeric(df.iloc[:, 14+1]) # 栏目14
        df_final[Config.COL_TOTAL_HV_COUNT]     = robust_to_numeric(df.iloc[:, 15+1]) # 栏目15
        df_final[Config.COL_TOTAL_HV_CAPACITY]  = robust_to_numeric(df.iloc[:, 16+1]) # 栏目16
        df_final[Config.COL_TOTAL_LV_COUNT]     = robust_to_numeric(df.iloc[:, 17+1]) # 栏目17
        df_final[Config.COL_TOTAL_LV_CAPACITY]  = robust_to_numeric(df.iloc[:, 18+1]) # 栏目18

        # 提取 完成销户 数据
        df_final[Config.COL_NEW_ALL_COUNT]      = robust_to_numeric(df.iloc[:, 19+1]) # 栏目19
        df_final[Config.COL_NEW_ALL_CAPACITY]   = robust_to_numeric(df.iloc[:, 20+1]) # 栏目20
        df_final[Config.COL_NEW_HV_COUNT]       = robust_to_numeric(df.iloc[:, 21+1]) # 栏目21
        df_final[Config.COL_NEW_HV_CAPACITY]    = robust_to_numeric(df.iloc[:, 22+1]) # 栏目22
        df_final[Config.COL_NEW_LV_COUNT]       = robust_to_numeric(df.iloc[:, 23+1]) # 栏目23
        df_final[Config.COL_NEW_LV_CAPACITY]    = robust_to_numeric(df.iloc[:, 24+1]) # 栏目24
        ### 重构/新增结束 ###

        return df_final
    except Exception as e:
        print(f"    [错误] 处理文件 {file_path.name} 时发生未知错误。")
        print(traceback.format_exc())
        return None

# ==============================================================================
# 3. 主流程 (Transformation & Loading)
# ==============================================================================
def main():
    print("开始执行数据处理流程...")
    all_dataframes = []
    if not Config.INPUT_DIR.exists():
        print(f"[致命错误] 输入文件夹不存在: {Config.INPUT_DIR}")
        return
    for file_path in sorted(Config.INPUT_DIR.glob('*.xlsx')):
        if re.match(Config.FILENAME_PATTERN, file_path.name):
            df = process_single_file(file_path)
            if df is not None:
                all_dataframes.append(df)
    if not all_dataframes:
        print("\n处理结束，但未从任何文件中成功提取数据。请检查上述错误日志。")
        return
    print("\n所有文件数据读取完毕，正在聚合成总表...")
    long_df = pd.concat(all_dataframes, ignore_index=True)

    long_df.fillna(0, inplace=True)

    ### 重构/新增开始 ###
    # 在聚合后的 long_df 上执行计算，生成所有“减容”相关的衍生指标
    print("正在计算所有衍生指标 (减容 = 减容销户 - 销户)...")

    # 计算 合计减容
    long_df[Config.COL_INC_ALL_COUNT]    = long_df[Config.COL_TOTAL_ALL_COUNT] - long_df[Config.COL_NEW_ALL_COUNT]
    long_df[Config.COL_INC_ALL_CAPACITY] = long_df[Config.COL_TOTAL_ALL_CAPACITY] - long_df[Config.COL_NEW_ALL_CAPACITY]

    # 计算 高压减容
    long_df[Config.COL_INC_HV_COUNT]     = long_df[Config.COL_TOTAL_HV_COUNT] - long_df[Config.COL_NEW_HV_COUNT]
    long_df[Config.COL_INC_HV_CAPACITY]  = long_df[Config.COL_TOTAL_HV_CAPACITY] - long_df[Config.COL_NEW_HV_CAPACITY]

    # 计算 低压减容
    long_df[Config.COL_INC_LV_COUNT]     = long_df[Config.COL_TOTAL_LV_COUNT] - long_df[Config.COL_NEW_LV_COUNT]
    long_df[Config.COL_INC_LV_CAPACITY]  = long_df[Config.COL_TOTAL_LV_CAPACITY] - long_df[Config.COL_NEW_LV_CAPACITY]
    ### 重构/新增结束 ###

    print("正在生成最终的多工作表Excel报表...")
    Config.OUTPUT_FILE.parent.mkdir(parents=True, exist_ok=True)
    with pd.ExcelWriter(Config.OUTPUT_FILE, engine='openpyxl') as writer:
        sheets_to_create = Config.get_sheets_to_create()
        # 现在的 sheets_to_create 已经包含了全部18个指标的工作表
        for sheet_name, value_col in sheets_to_create.items():
            print(f"  -> 正在生成工作表: {sheet_name}")
            try:
                pivot_df = long_df.pivot_table(
                    index=['序号', '行业'], columns='年月', values=value_col, aggfunc='sum')
                # 填充透视表中可能出现的NaN（例如某个月份某个行业没有数据）
                pivot_df.fillna(0, inplace=True)
                pivot_df.columns.name = None
                pivot_df = pivot_df.reset_index()
                pivot_df.rename(columns={'行业': '分类'}, inplace=True)
                pivot_df.to_excel(writer, sheet_name=sheet_name, index=False)
            except KeyError as e:
                print(f"    [警告] 在生成工作表 '{sheet_name}' 时找不到数据列 '{e}'，已跳过。")
                continue
    print(f"\n全部任务成功完成！结果已保存到: {Config.OUTPUT_FILE}")

if __name__ == '__main__':
    main()

开始执行数据处理流程...
  -> 正在处理文件: 201801.xlsx
  -> 正在处理文件: 201802.xlsx
  -> 正在处理文件: 201803.xlsx
  -> 正在处理文件: 201804.xlsx
  -> 正在处理文件: 201805.xlsx
  -> 正在处理文件: 201806.xlsx
  -> 正在处理文件: 201807.xlsx
  -> 正在处理文件: 201808.xlsx
  -> 正在处理文件: 201809.xlsx
  -> 正在处理文件: 201810.xlsx
  -> 正在处理文件: 201811.xlsx
  -> 正在处理文件: 201812.xlsx
  -> 正在处理文件: 201901.xlsx
  -> 正在处理文件: 201902.xlsx
  -> 正在处理文件: 201903.xlsx
  -> 正在处理文件: 201904.xlsx
  -> 正在处理文件: 201905.xlsx
  -> 正在处理文件: 201906.xlsx
  -> 正在处理文件: 201907.xlsx
  -> 正在处理文件: 201908.xlsx
  -> 正在处理文件: 201909.xlsx
  -> 正在处理文件: 201910.xlsx
  -> 正在处理文件: 201911.xlsx
  -> 正在处理文件: 201912.xlsx
  -> 正在处理文件: 202001.xlsx
  -> 正在处理文件: 202002.xlsx
  -> 正在处理文件: 202003.xlsx
  -> 正在处理文件: 202004.xlsx
  -> 正在处理文件: 202005.xlsx
  -> 正在处理文件: 202006.xlsx
  -> 正在处理文件: 202007.xlsx
  -> 正在处理文件: 202008.xlsx
  -> 正在处理文件: 202009.xlsx
  -> 正在处理文件: 202010.xlsx
  -> 正在处理文件: 202011.xlsx
  -> 正在处理文件: 202012.xlsx
  -> 正在处理文件: 202101.xlsx
  -> 正在处理文件: 202102.xlsx
  -> 正在处理文件: 202103.xlsx
  -> 正在处理文件

d:\Software\Anaconda\envs\dianli\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
d:\Software\Anaconda\envs\dianli\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


  -> 正在处理文件: 202404.xlsx
  -> 正在处理文件: 202405.xlsx
  -> 正在处理文件: 202406.xlsx
  -> 正在处理文件: 202407.xlsx
  -> 正在处理文件: 202408.xlsx
  -> 正在处理文件: 202409.xlsx
  -> 正在处理文件: 202410.xlsx


d:\Software\Anaconda\envs\dianli\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


  -> 正在处理文件: 202411.xlsx
  -> 正在处理文件: 202412.xlsx
  -> 正在处理文件: 202501.xlsx
  -> 正在处理文件: 202502.xlsx
  -> 正在处理文件: 202503.xlsx
  -> 正在处理文件: 202504.xlsx
  -> 正在处理文件: 202505.xlsx
  -> 正在处理文件: 202506.xlsx
  -> 正在处理文件: 202507.xlsx
  -> 正在处理文件: 202508.xlsx


d:\Software\Anaconda\envs\dianli\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
d:\Software\Anaconda\envs\dianli\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


  -> 正在处理文件: 202509.xlsx
  -> 正在处理文件: 202510.xlsx
  -> 正在处理文件: 202511.xlsx

所有文件数据读取完毕，正在聚合成总表...
正在计算所有衍生指标 (减容 = 减容销户 - 销户)...
正在生成最终的多工作表Excel报表...
  -> 正在生成工作表: 完成减容销户_户数
  -> 正在生成工作表: 完成减容销户_容量
  -> 正在生成工作表: 高压完成减容销户_户数
  -> 正在生成工作表: 高压完成减容销户_容量
  -> 正在生成工作表: 低压完成减容销户_户数
  -> 正在生成工作表: 低压完成减容销户_容量
  -> 正在生成工作表: 完成销户_户数
  -> 正在生成工作表: 完成销户_容量
  -> 正在生成工作表: 高压完成销户_户数
  -> 正在生成工作表: 高压完成销户_容量
  -> 正在生成工作表: 低压完成销户_户数
  -> 正在生成工作表: 低压完成销户_容量
  -> 正在生成工作表: 完成减容_户数
  -> 正在生成工作表: 完成减容_容量
  -> 正在生成工作表: 高压完成减容_户数
  -> 正在生成工作表: 高压完成减容_容量
  -> 正在生成工作表: 低压完成减容_户数
  -> 正在生成工作表: 低压完成减容_容量

全部任务成功完成！结果已保存到: E:\A智网\业扩分析\12月分析\11月业扩月度报告\25年11月业扩报告_减容销户业扩.xlsx


In [3]:
#月度综合报告添加近三月的净增容量对比变化
#添加了累计增容和累计增速11.13
#将前期提出的新装增容、减容销户基本数据整合分析，得出所需月份的分析表文件'x月综合分析报告'，单位：万千伏安、户

import pandas as pd
from pathlib import Path
from typing import Dict, List, Optional

# ==============================================================================
# 1. 配置层 (Configuration Layer)
# ==============================================================================
class Config:
    """集中管理所有分析报告的配置参数"""
    # --- 输入路径和文件 ---
    BASE_PATH = Path(r'E:\A智网\业扩分析\12月分析\11月业扩月度报告')
    FILE_NEW_INC = BASE_PATH / '25年11月业扩报告_新装增容业扩.xlsx'
    FILE_DEC_TERM = BASE_PATH / '25年11月业扩报告_减容销户业扩.xlsx'

    # --- 输出文件 ---
    OUTPUT_FILE = BASE_PATH / '11月业扩月度综合分析报告v4.xlsx'

    # --- 分析周期定义 ---
    TARGET_MONTH = '202511'

    # --- 容量单位转换配置 ---
    CAPACITY_UNIT_CONVERSION_FACTOR = 10000
    CAPACITY_DECIMAL_PLACES = 3

    # --- 指标所在的工作表名称 ---
    SHEET_NEW_COUNT = '完成新装_户数'
    SHEET_NEW_CAP = '完成新装_容量'
    SHEET_INC_COUNT = '完成增容_户数'
    SHEET_INC_CAP = '完成增容_容量'
    SHEET_TOTAL_INC_CAP = '完成新装增容_容量'
    SHEET_DEC_COUNT = '完成减容_户数'
    SHEET_DEC_CAP = '完成减容_容量'
    SHEET_TERM_COUNT = '完成销户_户数'
    SHEET_TERM_CAP = '完成销户_容量'
    SHEET_TOTAL_DEC_CAP = '完成减容销户_容量'

    # --- 输出报告的列名 ---
    COL_SEQ = '序号'
    COL_CATEGORY = '分类'
    COL_NEW_COMBINED = '本月新装(户/万kVA)'
    COL_INC_COMBINED = '本月增容(户/-万kVA)'
    COL_DEC_COMBINED = '本月减容(户/万kVA)'
    COL_TERM_COMBINED = '本月销户(户/万kVA)'
    COL_NET_INC_MONTH = '本月净增容量(万kVA)'
    COL_NET_INC_YTD = '本月累计净增容量(万kVA)'
    COL_NET_INC_MOM = '净增容量环比'
    COL_NET_INC_YOY = '净增容量同比'
    COL_NET_INC_YTD_YOY = '本月累计净增容量同比'
    COL_NET_INC_LAST_3_MONTHS = '近三月净增容量(万kVA)'
    COL_NET_INC_LAST_3_MONTHS_YOY = '近三月净增容量同比'


# ==============================================================================
# 2. 辅助函数层 (Helper Functions)
# ==============================================================================
def read_single_indicator(file_path: Path, sheet_name: str, cols_to_read: List[str]) -> Optional[pd.DataFrame]:
    """从单个Excel文件的指定工作表中读取特定列"""
    print(f"  -> 正在读取: 文件 '{file_path.name}', 工作表 '{sheet_name}'")
    if not file_path.exists():
        print(f"    [错误] 文件不存在: {file_path}")
        return None
    try:
        df = pd.read_excel(file_path, sheet_name=sheet_name) # <--- 修改：先读取所有列
        # 检查所需列是否存在
        missing_cols = [col for col in cols_to_read if col not in df.columns]
        if missing_cols:
            print(f"    [警告] 工作表 '{sheet_name}' 中缺少列: {', '.join(missing_cols)}")
            # 过滤掉不存在的列，继续处理
            cols_to_read = [col for col in cols_to_read if col in df.columns]

        df = df[cols_to_read] # 只保留需要的列
        df['分类'] = df['分类'].astype(str)
        return df.set_index(Config.COL_CATEGORY)
    except (FileNotFoundError, ValueError) as e:
        print(f"    [错误] 读取失败: {e}")
        return None
    except Exception as e:
        print(f"    [严重错误] 发生未知错误: {e}")
        return None

def calculate_percentage_change(current, previous):
    """健壮的百分比变化计算函数，处理除以零的情况"""
    if pd.isna(current) or pd.isna(previous):
        return 'N/A'
    if previous == 0:
        return 'N/A'
    change = (current - previous) / abs(previous)
    return f"{change:.2%}"


# ==============================================================================
# 3. 主流程 (Main Logic)
# ==============================================================================
def main():
    """主执行函数"""
    print("开始生成业扩月度综合分析报告...")

    # --- 1. 动态计算所有相关月份 ---
    try:
        current_year = int(Config.TARGET_MONTH[:4])
        current_month_num = int(Config.TARGET_MONTH[4:])
        
        target_month_col = Config.TARGET_MONTH
        previous_month_dt = pd.to_datetime(target_month_col, format='%Y%m') - pd.DateOffset(months=1)
        previous_month_col = previous_month_dt.strftime('%Y%m')
        last_year_month_col = f"{current_year-1}{current_month_num:02d}"
        
        # 本年累计所需月份
        ytd_months_cols_current = [f"{current_year}{m:02d}" for m in range(1, current_month_num + 1)]
        
        # 【新增】去年同期累计所需月份
        ytd_months_cols_previous = [f"{current_year-1}{m:02d}" for m in range(1, current_month_num + 1)]
        
        # 近三月指标所需月份
        last_3_months_current_year = [(pd.to_datetime(target_month_col, format='%Y%m') - pd.DateOffset(months=i)).strftime('%Y%m') for i in range(3)]
        last_3_months_previous_year = [str(int(m[:4]) - 1) + m[4:] for m in last_3_months_current_year]

        # 汇总所有需要从Excel读取的列
        all_required_cols = list(set(
            [Config.COL_SEQ, Config.COL_CATEGORY, target_month_col, previous_month_col, last_year_month_col] 
            + ytd_months_cols_current 
            + ytd_months_cols_previous # <--- 新增
            + last_3_months_current_year 
            + last_3_months_previous_year
        ))
        
        print(f"分析月份: {target_month_col}")
        print(f"今年累计月份: {ytd_months_cols_current}")
        print(f"去年累计月份: {ytd_months_cols_previous}")

    except ValueError:
        print(f"[致命错误] 'TARGET_MONTH' ({Config.TARGET_MONTH}) 格式不正确。")
        return

    # --- 2. 读取核心数据 ---
    df_total_inc = read_single_indicator(Config.FILE_NEW_INC, Config.SHEET_TOTAL_INC_CAP, all_required_cols)
    df_total_dec = read_single_indicator(Config.FILE_DEC_TERM, Config.SHEET_TOTAL_DEC_CAP, all_required_cols)

    if df_total_inc is None or df_total_dec is None:
        print("\n[致命错误] 无法读取核心数据，程序终止。")
        return

    net_increase_df = df_total_inc.drop(columns=[Config.COL_SEQ], errors='ignore').subtract(df_total_dec.drop(columns=[Config.COL_SEQ], errors='ignore'), fill_value=0)

    # --- 3. 计算所有衍生指标 ---
    print("\n正在计算所有衍生指标...")
    summary_df = pd.DataFrame(index=net_increase_df.index)
    
    # a. 计算原有指标
    summary_df[Config.COL_NET_INC_MONTH] = (net_increase_df.get(target_month_col, 0) / Config.CAPACITY_UNIT_CONVERSION_FACTOR).round(Config.CAPACITY_DECIMAL_PLACES)
    
    # 今年累计
    net_inc_ytd_current = net_increase_df[ytd_months_cols_current].sum(axis=1)
    summary_df[Config.COL_NET_INC_YTD] = (net_inc_ytd_current / Config.CAPACITY_UNIT_CONVERSION_FACTOR).round(Config.CAPACITY_DECIMAL_PLACES)
    
    net_inc_current = net_increase_df.get(target_month_col, 0)
    net_inc_previous = net_increase_df.get(previous_month_col, 0)
    net_inc_last_year = net_increase_df.get(last_year_month_col, 0)
    summary_df[Config.COL_NET_INC_MOM] = [calculate_percentage_change(cur, prev) for cur, prev in zip(net_inc_current, net_inc_previous)]
    summary_df[Config.COL_NET_INC_YOY] = [calculate_percentage_change(cur, last) for cur, last in zip(net_inc_current, net_inc_last_year)]
    
    # b. 计算近三月指标
    net_inc_last_3_months_current = net_increase_df[last_3_months_current_year].sum(axis=1)
    summary_df[Config.COL_NET_INC_LAST_3_MONTHS] = (net_inc_last_3_months_current / Config.CAPACITY_UNIT_CONVERSION_FACTOR).round(Config.CAPACITY_DECIMAL_PLACES)
    
    net_inc_last_3_months_previous = net_increase_df[last_3_months_previous_year].sum(axis=1)
    
    summary_df[Config.COL_NET_INC_LAST_3_MONTHS_YOY] = [
        calculate_percentage_change(cur, prev) 
        for cur, prev in zip(net_inc_last_3_months_current, net_inc_last_3_months_previous)
    ]
    
    # c. 【新增】计算本月止累计净增容量同比
    #    去年同期累计
    net_inc_ytd_previous = net_increase_df[ytd_months_cols_previous].sum(axis=1)
    
    #    计算同比
    summary_df[Config.COL_NET_INC_YTD_YOY] = [
        calculate_percentage_change(cur, prev)
        for cur, prev in zip(net_inc_ytd_current, net_inc_ytd_previous)
    ]
    
    # --- 4. 读取并组合户数/容量数据 ---
    # (这部分逻辑完全不变)
    print("\n正在读取并组合 '户数/容量' 数据...")
    # ... (省略) ...
    
    # --- 5. 整理并输出最终报告 ---
    print("\n正在生成最终报告...")
    
    if Config.COL_SEQ in df_total_inc.columns:
        summary_df = summary_df.join(df_total_inc[[Config.COL_SEQ]])
    else:
        summary_df[Config.COL_SEQ] = 'N/A'

    final_df = summary_df.reset_index()

    # 【已更新】调整最终的列顺序，加入新指标
    final_cols_order = [
        Config.COL_SEQ,
        Config.COL_CATEGORY,
        Config.COL_NEW_COMBINED,
        Config.COL_INC_COMBINED,
        Config.COL_DEC_COMBINED,
        Config.COL_TERM_COMBINED,
        Config.COL_NET_INC_MONTH, #本月净增容
        Config.COL_NET_INC_YTD, # 累计净增容
        Config.COL_NET_INC_LAST_3_MONTHS, #近三月净增容
        Config.COL_NET_INC_YTD_YOY, # 累计净增容同比
        Config.COL_NET_INC_LAST_3_MONTHS_YOY, #近三月净增容同比
        Config.COL_NET_INC_MOM,  #环比
        Config.COL_NET_INC_YOY,  #同比
    ]
    
    # 健壮性处理：只保留实际存在的列，以防某些列计算失败
    final_df = final_df[[col for col in final_cols_order if col in final_df.columns]]

    try:
        Config.OUTPUT_FILE.parent.mkdir(parents=True, exist_ok=True)
        final_df.to_excel(Config.OUTPUT_FILE, sheet_name='月度综合分析', index=False)
        print(f"\n全部任务成功完成！结果已保存到: {Config.OUTPUT_FILE}")
    except Exception as e:
        print(f"\n[致命错误] 保存Excel文件失败: {e}")

if __name__ == '__main__':
    main()

开始生成业扩月度综合分析报告...
分析月份: 202511
今年累计月份: ['202501', '202502', '202503', '202504', '202505', '202506', '202507', '202508', '202509', '202510', '202511']
去年累计月份: ['202401', '202402', '202403', '202404', '202405', '202406', '202407', '202408', '202409', '202410', '202411']
  -> 正在读取: 文件 '25年11月业扩报告_新装增容业扩.xlsx', 工作表 '完成新装增容_容量'
  -> 正在读取: 文件 '25年11月业扩报告_减容销户业扩.xlsx', 工作表 '完成减容销户_容量'

正在计算所有衍生指标...

正在读取并组合 '户数/容量' 数据...

正在生成最终报告...

全部任务成功完成！结果已保存到: E:\A智网\业扩分析\12月分析\11月业扩月度报告\11月业扩月度综合分析报告v4.xlsx


In [4]:
import pandas as pd
from pathlib import Path
import traceback

# ==============================================================================
# 1. 配置层 (Configuration Layer)
# ==============================================================================
class Config:
    """集中管理所有分析报告的配置参数"""
    # --- 输入路径和文件 ---
    BASE_PATH = Path(r'E:\A智网\业扩分析\12月分析\11月业扩月度报告')
    FILE_NEW_INC = BASE_PATH / '25年11月业扩报告_新装增容业扩.xlsx'
    FILE_DEC_TERM = BASE_PATH / '25年11月业扩报告_减容销户业扩.xlsx'

    # --- 输出文件 ---
    # 我们将结果写入一个新的专用文件中，以防覆盖原有报告
    OUTPUT_FILE = BASE_PATH / '业扩月度累计净增容量趋势分析.xlsx'

    # --- 指标所在的工作表名称 ---
    SHEET_TOTAL_INC_CAP = '完成新装增容_容量'
    SHEET_TOTAL_DEC_CAP = '完成减容销户_容量'

    # --- 关键列名 ---
    COL_SEQ = '序号'
    COL_CATEGORY = '分类'
    
    # --- 输出Sheet和列名 ---
    OUTPUT_SHEET_YTD_CAP = '累计净增容量'
    OUTPUT_SHEET_YTD_YOY = '累计净增容量同比'
    
    # --- 容量单位转换 ---
    CAPACITY_UNIT_CONVERSION_FACTOR = 10000  # 从 kVA 转换为 万kVA
    CAPACITY_DECIMAL_PLACES = 3

# ==============================================================================
# 2. 辅助函数层 (Helper Functions)
# ==============================================================================
def calculate_percentage_change(current, previous):
    """健壮的百分比变化计算函数"""
    if pd.isna(current) or pd.isna(previous):
        return 'N/A'
    if previous == 0:
        return 'N/A' if current != 0 else '0.00%'
    change = (current - previous) / abs(previous)
    return f"{change:.2%}"

# ==============================================================================
# 3. 主流程 (Main Logic)
# ==============================================================================
def main():
    """主执行函数"""
    print("--- 开始生成累计净增容量趋势分析报告 ---")

    try:
        # --- 1. 读取并准备核心数据 ---
        print("正在读取核心数据文件...")
        df_inc = pd.read_excel(Config.FILE_NEW_INC, sheet_name=Config.SHEET_TOTAL_INC_CAP)
        df_inc.columns = df_inc.columns.map(str)
        
        df_dec = pd.read_excel(Config.FILE_DEC_TERM, sheet_name=Config.SHEET_TOTAL_DEC_CAP)
        df_dec.columns = df_dec.columns.map(str)

        id_cols = [Config.COL_SEQ, Config.COL_CATEGORY]
        month_cols_inc = [col for col in df_inc.columns if col.isdigit() and len(col) == 6]
        month_cols_dec = [col for col in df_dec.columns if col.isdigit() and len(col) == 6]
        all_month_cols = sorted(list(set(month_cols_inc + month_cols_dec)))

        # 将'分类'设为索引，方便计算
        # 【修正】在设置索引前，先确保'分类'列存在
        if Config.COL_CATEGORY not in df_inc.columns or Config.COL_CATEGORY not in df_dec.columns:
            print(f"[致命错误] 源文件中缺少关键列 '{Config.COL_CATEGORY}'。")
            return
        df_inc.set_index(Config.COL_CATEGORY, inplace=True)
        df_dec.set_index(Config.COL_CATEGORY, inplace=True)
        
        net_increase_df = df_inc[month_cols_inc].subtract(df_dec[month_cols_dec], fill_value=0)
        print("每月净增容量计算完成。")

        # --- 2. 循环计算每个月的累计值和同比 ---
        print("正在循环计算每个月的累计指标...")
        ytd_capacity_results = {}
        ytd_yoy_results = {}

        for month_str in all_month_cols:
            try:
                current_year = int(month_str[:4])
                current_month_num = int(month_str[4:])
                
                ytd_cols_current = [f"{current_year}{m:02d}" for m in range(1, current_month_num + 1)]
                ytd_cols_current = [col for col in ytd_cols_current if col in net_increase_df.columns]
                
                ytd_cols_previous = [f"{current_year - 1}{m:02d}" for m in range(1, current_month_num + 1)]
                ytd_cols_previous = [col for col in ytd_cols_previous if col in net_increase_df.columns]

                if not ytd_cols_current:
                    continue

                ytd_cap_current = net_increase_df[ytd_cols_current].sum(axis=1)
                ytd_capacity_results[month_str] = (ytd_cap_current / Config.CAPACITY_UNIT_CONVERSION_FACTOR).round(Config.CAPACITY_DECIMAL_PLACES)

                if ytd_cols_previous:
                    ytd_cap_previous = net_increase_df[ytd_cols_previous].sum(axis=1)
                    yoy_series = pd.Series(
                        [calculate_percentage_change(cur, prev) for cur, prev in zip(ytd_cap_current, ytd_cap_previous)],
                        index=net_increase_df.index
                    )
                    ytd_yoy_results[month_str] = yoy_series
                else:
                    ytd_yoy_results[month_str] = 'N/A'
            
            except Exception as e:
                print(f"  [警告] 处理月份 {month_str} 时出错: {e}")
                continue
        
        print("所有月份累计指标计算完成。")

        # --- 3. 组装最终报告 ---
        print("\n正在组装最终报告...")
        report_ytd_cap = pd.DataFrame(ytd_capacity_results)
        report_ytd_yoy = pd.DataFrame(ytd_yoy_results)
        
        # 提取包含'序号'和'分类'的标识DataFrame
        # 【修正】从原始df_inc中提取，因为它没有被修改索引
        id_df = pd.read_excel(Config.FILE_NEW_INC, sheet_name=Config.SHEET_TOTAL_INC_CAP)[id_cols]
        id_df.columns = id_df.columns.map(str) # 确保列名是字符串
        
        # 【核心修正】使用正确的列名进行合并
        # reset_index() 会创建一个名为'分类'的列
        report_ytd_cap = pd.merge(id_df, report_ytd_cap.reset_index(), left_on=Config.COL_CATEGORY, right_on=Config.COL_CATEGORY)
        report_ytd_yoy = pd.merge(id_df, report_ytd_yoy.reset_index(), left_on=Config.COL_CATEGORY, right_on=Config.COL_CATEGORY)

        # --- 4. 写入Excel ---
        print(f"正在将结果写入到文件: {Config.OUTPUT_FILE}")
        with pd.ExcelWriter(Config.OUTPUT_FILE, engine='openpyxl') as writer:
            report_ytd_cap.to_excel(writer, sheet_name=Config.OUTPUT_SHEET_YTD_CAP, index=False)
            print(f"  -> 已写入Sheet: '{Config.OUTPUT_SHEET_YTD_CAP}'")
            
            report_ytd_yoy.to_excel(writer, sheet_name=Config.OUTPUT_SHEET_YTD_YOY, index=False)
            print(f"  -> 已写入Sheet: '{Config.OUTPUT_SHEET_YTD_YOY}'")

        print(f"\n--- 全部任务成功完成！结果已保存到: {Config.OUTPUT_FILE} ---")

    except FileNotFoundError:
        print(f"[致命错误] 无法找到输入文件，请检查路径: {Config.FILE_NEW_INC} 或 {Config.FILE_DEC_TERM}")
    except Exception as e:
        print(f"[致命错误] 执行主流程时发生未知错误。")
        print(traceback.format_exc())

if __name__ == '__main__':
    main()

--- 开始生成累计净增容量趋势分析报告 ---
正在读取核心数据文件...
每月净增容量计算完成。
正在循环计算每个月的累计指标...
所有月份累计指标计算完成。

正在组装最终报告...
正在将结果写入到文件: E:\A智网\业扩分析\12月分析\11月业扩月度报告\业扩月度累计净增容量趋势分析.xlsx
  -> 已写入Sheet: '累计净增容量'
  -> 已写入Sheet: '累计净增容量同比'

--- 全部任务成功完成！结果已保存到: E:\A智网\业扩分析\12月分析\11月业扩月度报告\业扩月度累计净增容量趋势分析.xlsx ---


In [5]:
import pandas as pd
from pathlib import Path
import numpy as np
import traceback

# ==============================================================================
# 1. 配置层 (Configuration Layer)
# ==============================================================================
class Config:
    """集中管理所有文件路径、Sheet名和列名"""
    # --- 【请修改】文件路径和目标月份 ---
    # 注意：输入和输出是同一个文件
    FILE_PATH = Path(r'E:\A智网\业扩分析\12月分析\业扩月度电量.xlsx')
    
    # !! 关键 !!: 每次运行时，只需要修改这里的目标月份
    TARGET_MONTH = '202511'

    # --- Sheet名称 ---
    INPUT_SHEET_NAME = '电量'
    OUTPUT_SHEET_NAME = '同比结果'

    # --- 关键列名 ---
    SEQ_COL = '序号'
    CATEGORY_COL = '分类'
    
    # --- 输出列名 ---
    OUTPUT_COL_MONTHLY_YOY = '当月电量同比'
    OUTPUT_COL_3MONTH_YOY = '近三月电量同比'

# ==============================================================================
# 2. 辅助函数层 (Helper Functions)
# ==============================================================================
def calculate_percentage_change(current, previous):
    """健壮的百分比变化计算函数，处理除以零或数据缺失的情况"""
    # 如果任一值为NaN或None，则无法计算
    if pd.isna(current) or pd.isna(previous):
        return np.nan # 返回NaN，以便后续可以格式化为'N/A'
    
    # 如果去年同期值为0
    if previous == 0:
        # 如果今年值也为0，则无变化
        if current == 0:
            return 0.0
        # 如果今年值不为0，则为无穷大增长，通常表示为'N/A'或特殊标记
        else:
            return np.inf # 返回无穷大
            
    change = (current - previous) / abs(previous)
    return change

# =' '='============================================================================
# 3. 主流程 (Main Workflow)
# ==============================================================================
def main():
    """主执行函数"""
    print(f"--- 开始为月份 {Config.TARGET_MONTH} 计算电量同比 ---")
    print(f"--- 处理文件: {Config.FILE_PATH} ---")

    if not Config.FILE_PATH.exists():
        print(f"[致命错误] 输入文件不存在: {Config.FILE_PATH}")
        return

    try:
        # 1. 加载数据
        print(f"正在加载Sheet '{Config.INPUT_SHEET_NAME}'...")
        df = pd.read_excel(Config.FILE_PATH, sheet_name=Config.INPUT_SHEET_NAME)
        
        # --- 【核心修正】将所有列名强制转换为字符串类型 ---
        df.columns = df.columns.map(str)
        print("  -> 已将所有列名转换为字符串格式以确保匹配。")
        
        # 2. 动态计算所有相关月份的列名
        print("正在计算相关月份...")
        current_year = int(Config.TARGET_MONTH[:4])
        current_month_num = int(Config.TARGET_MONTH[4:])
        
        target_month_col = Config.TARGET_MONTH
        last_year_month_col = f"{current_year - 1}{current_month_num:02d}"
        
        last_3_months_current_year = []
        for i in range(3):
            month_dt = pd.to_datetime(target_month_col, format='%Y%m') - pd.DateOffset(months=i)
            last_3_months_current_year.append(month_dt.strftime('%Y%m'))
        
        last_3_months_previous_year = [str(int(m[:4]) - 1) + m[4:] for m in last_3_months_current_year]

        print(f"  - 当月: {target_month_col}, 上年同期: {last_year_month_col}")
        print(f"  - 今年近三月: {last_3_months_current_year}")
        print(f"  - 去年同期近三月: {last_3_months_previous_year}")

        # 3. 检查所有需要的列是否存在
        all_required_cols = [target_month_col, last_year_month_col] + last_3_months_current_year + last_3_months_previous_year
        missing_cols = [col for col in all_required_cols if col not in df.columns]
        if missing_cols:
            print(f"\n[严重警告] 源数据Sheet '{Config.INPUT_SHEET_NAME}' 中缺少以下必需的月份列: {', '.join(sorted(list(set(missing_cols))))}")
            print("程序将继续运行，但缺少数据的计算结果将为空。")
            for col in missing_cols:
                df[col] = np.nan

        # 4. 创建结果DataFrame
        result_df = df[[Config.SEQ_COL, Config.CATEGORY_COL]].copy()

        # 5. 计算“当月电量同比”
        print("\n正在计算“当月电量同比”...")
        current_month_load = pd.to_numeric(df[target_month_col], errors='coerce')
        last_year_month_load = pd.to_numeric(df[last_year_month_col], errors='coerce')
        result_df[Config.OUTPUT_COL_MONTHLY_YOY] = [
            calculate_percentage_change(cur, prev) 
            for cur, prev in zip(current_month_load, last_year_month_load)
        ]
        
        # 6. 计算“近三月电量同比”
        print("正在计算“近三月电量同比”...")
        current_3m_sum = df[last_3_months_current_year].apply(pd.to_numeric, errors='coerce').sum(axis=1, skipna=False)
        previous_3m_sum = df[last_3_months_previous_year].apply(pd.to_numeric, errors='coerce').sum(axis=1, skipna=False)
        
        result_df[Config.OUTPUT_COL_3MONTH_YOY] = [
            calculate_percentage_change(cur, prev)
            for cur, prev in zip(current_3m_sum, previous_3m_sum)
        ]
        
        # 7. 格式化百分比输出
        print("正在格式化输出结果...")
        for col in [Config.OUTPUT_COL_MONTHLY_YOY, Config.OUTPUT_COL_3MONTH_YOY]:
            result_df[col] = result_df[col].apply(lambda x: f"{x:.2%}" if pd.notna(x) and np.isfinite(x) else 'N/A')

        # 8. 将结果写入到原始Excel文件的新Sheet中
        print(f"正在将结果写入到文件 '{Config.FILE_PATH.name}' 的新Sheet '{Config.OUTPUT_SHEET_NAME}'...")
        with pd.ExcelWriter(Config.FILE_PATH, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
            result_df.to_excel(writer, sheet_name=Config.OUTPUT_SHEET_NAME, index=False)
        
        print(f"\n--- 全部任务成功完成！ ---")
        print(f"请打开文件 '{Config.FILE_PATH}' 并检查名为 '{Config.OUTPUT_SHEET_NAME}' 的工作表。")

    except FileNotFoundError:
        print(f"[致命错误] 无法找到输入文件: {Config.FILE_PATH}")
    except ValueError as e:
        if f"Worksheet {Config.INPUT_SHEET_NAME} does not exist" in str(e):
             print(f"[致命错误] 在Excel文件中找不到名为 '{Config.INPUT_SHEET_NAME}' 的工作表。")
        else:
            print(f"[致命错误] 执行过程中发生值错误: {e}")
            print(traceback.format_exc())
    except Exception as e:
        print(f"[致命错误] 执行过程中发生未知错误。")
        print(traceback.format_exc())

if __name__ == '__main__':
    main()

--- 开始为月份 202511 计算电量同比 ---
--- 处理文件: E:\A智网\业扩分析\12月分析\业扩月度电量.xlsx ---
正在加载Sheet '电量'...
  -> 已将所有列名转换为字符串格式以确保匹配。
正在计算相关月份...
  - 当月: 202511, 上年同期: 202411
  - 今年近三月: ['202511', '202510', '202509']
  - 去年同期近三月: ['202411', '202410', '202409']

正在计算“当月电量同比”...
正在计算“近三月电量同比”...
正在格式化输出结果...
正在将结果写入到文件 '业扩月度电量.xlsx' 的新Sheet '同比结果'...

--- 全部任务成功完成！ ---
请打开文件 'E:\A智网\业扩分析\12月分析\业扩月度电量.xlsx' 并检查名为 '同比结果' 的工作表。
